# Subset Analysis -- Entity

In [1]:
import spacy
nlp = spacy.load("en_core_web_sm")

def count_entities(text):
    doc = nlp(text)
    return len([ent for ent in doc.ents])

In [8]:
raw = open("val_candidates_constrained_top1_with_rougeL.json").read()

# 找到最后一个 '}' —— 这是最后一个完整对象的结束
last_obj_end = raw.rfind("}")

# 截断文件到这个位置
trimmed = raw[:last_obj_end+1]

# 保证 list 有结尾的 ']'
if not trimmed.strip().endswith("]"):
    trimmed = trimmed + "\n]"

# 尝试解析
import json
try:
    data = json.loads(trimmed)
    print("修复成功！共载入对象数量：", len(data))
except Exception as e:
    print("修复失败：", e)


修复成功！共载入对象数量： 1000


In [10]:
from tqdm import tqdm
import json

with open("val_candidates_constrained_top1_with_rougeL.json") as f:
    data = json.load(f)

for item in tqdm(data):   # 加 tqdm！
    item["entity_count"] = count_entities(item["document"])



100%|██████████| 1000/1000 [04:40<00:00,  3.57it/s]


In [11]:
import numpy as np

entity_counts = np.array([x["entity_count"] for x in data])

print("Min entity count:", entity_counts.min())
print("Max entity count:", entity_counts.max())
print("Mean entity count:", entity_counts.mean())
print("Median entity count:", np.median(entity_counts))

print("33% percentile:", np.percentile(entity_counts, 33))
print("66% percentile:", np.percentile(entity_counts, 66))

Min entity count: 4
Max entity count: 3266
Mean entity count: 188.735
Median entity count: 134.0
33% percentile: 96.67000000000002
66% percentile: 182.34000000000003


In [13]:
entity_counts = [x["entity_count"] for x in data]
low_thr  = np.percentile(entity_counts, 33)
high_thr = np.percentile(entity_counts, 66)

def group(item):
    if item["entity_count"] <= low_thr:
        return "low"
    elif item["entity_count"] >= high_thr:
        return "high"
    else:
        return "medium"

for item in data:
    item["bucket"] = group(item)


In [14]:
from collections import defaultdict

bucket_scores = defaultdict(list)

for item in data:
    bucket_scores[item["bucket"]].append(item["rougeL_f1"])

for b in ["low","medium","high"]:
    print(b, np.mean(bucket_scores[b]))


low 0.18245699360139023
medium 0.16381951229560315
high 0.15723095936970397


# Successful vs Failed Cases

In [23]:
import json
import numpy as np
from tqdm import tqdm

# ---------- Load ----------
with open("/content/val_candidates_top1_with_rougeL_nli_sas.json") as f:
    data = json.load(f)

# ---------- Scoring Function ----------
def combined_score(x):
    return 0.5 * x["rougeL_f1"] + 0.3 * x["sas_score"] + 0.2 * x["nli_score"]

# ---------- Compute scores with tqdm ----------
for item in tqdm(data, desc="Computing combined scores"):
    item["combined"] = combined_score(item)

# ---------- Sort ----------
data_sorted = sorted(data, key=lambda x: x["combined"], reverse=True)

# ---------- Select top/bottom ----------
top_k = int(len(data) * 0.05)
success_cases = data_sorted[:top_k]
failure_cases = data_sorted[-top_k:]

# ---------- Pick best/worst ----------
best_case = success_cases[0]
worst_case = failure_cases[0]

print("Best Case ID:", best_case["id"])
print("Worst Case ID:", worst_case["id"])



Computing combined scores: 100%|██████████| 1000/1000 [00:00<00:00, 1700853.20it/s]

Best Case ID: 862
Worst Case ID: 560


In [27]:
# ---- 打印 Best / Worst Case ----
best_id = 862
worst_id = 560

def print_case(case_id):
    for item in data:
        if item["id"] == case_id:
            print("\n==============================")
            print(f"🔵 Case ID: {case_id}")
            print("==============================")
            print("Document:\n", item["document"][:800], "...\n")
            print("Reference Summary:\n", item["reference"], "\n")
            print("Chosen Summary:\n", item["candidates"][0], "\n")
            print("Scores:")
            print("  Rouge-L:", item["rougeL_f1"])
            print("  FactScore/NLI:", item["nli_score"])
            print("  SAS:", item["sas_score"])
            print("==============================\n")
            return
    print("❌ ID not found:", case_id)

# 打印两个 case
print_case(best_id)
print_case(worst_id)



🔵 Case ID: 862
Document:
 BOGOTA, Colombia (AP) — The Latest on the deadly river overflow in southern Colombia (all times local): 
 
 11:50 a.m. 
 
 Colombia President Juan Manuel Santos says at least 112 people have been killed after intense rains triggered an avalanche of mud and water from overflowing rivers that swept through a small city. 
 
 Santos arrived at the disaster zone Saturday, warning the death toll could rise as the search for survivors continues. 
 
 The incident happened around midnight in Mocoa, a city of 350,000 located near Colombia's border with Ecuador. 
 
 A surgeon at the local hospital says he believes there are at least 300 people injured and that doctors are running out of blood. 
 
 Witnesses described feeling buildings vibrate and say there was little time to seek refuge, catching som ...

Reference Summary:
 – Colombia President Juan Manuel Santos says at least 154 people have been killed after intense rains triggered an avalanche of mud and water from 